# Benchmarking Our Algorithm against others

In [29]:
import sys
sys.path.append('../')
from panav.environment.env import MultiTunnelEnv
from panav.hybrid import HybridGraph
from panav.util import flowtime,makespan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP
from panav.sequential import sequential_planning
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning



from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:

bloating_r = 0.5
vmax = 1.0

n_agents = list(range(2,21,2))

TIMEOUT = 100 # 1 minute timeout

# Algorithms
algs = {"S2M2": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "PBS_Tube": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Tube_Planning")[0],
        "TAHP": lambda hg: TAHP(hg,vmax=vmax,bloating_r=bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Tube": lambda hg: sequential_planning(Tube_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Temp_Obs": lambda hg: sequential_planning(Simple_MILP_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT)}

timeouted = {alg:False for alg in algs}

max_agents = {
        "S2M2": 21,
        "PBS_Tube": 21,
        "TAHP": 21,
        "Seq_Tube": 21,
        "Seq_Temp_Obs": 21
}

results = []

env_name = "MultiTunnel"

# Execution timeout handling
import signal
# Register an handler for the timeout
def timeout_handler(signum, frame, name,N):
    timeouted[name] = True
    print("Time out for algorithm", name, "N agent = ", N)
    raise TimeoutError
 

# Experiments
for i in range(len(n_agents)):
    N = n_agents[i]
    env = MultiTunnelEnv(n_tunnel = 4, tunnel_width = bloating_r*2.5, N_agent = N)
    HG = HybridGraph(env,bloating_r)

    for name,alg in algs.items():

        if N>max_agents[name] or timeouted[name]:
            continue
        
        print("Alg:", name, "N agent:",N)

        result = {}
        
        t0 = time()
        plan = alg(HG)
        t = time()-t0
        
        result['plan'] = plan
        result['time'] = t

        result['N_agent'] = N
        result['alg_name'] = name

        results.append(result)

        
        if t>TIMEOUT:
            timeouted[name] = True
            print("Time out for algorithm", name, "N agent = ", N)


        
        

    # Save checkpoint data
    with open("./data/{}.pkl".format(env_name),'wb') as fp:
        pkl.dump(results,fp)


/Users/tianpengzhang/opt/miniconda3/envs/PA-Nav/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


Alg: S2M2 N agent: 2
Alg: PBS_Tube N agent: 2
Alg: TAHP N agent: 2
Alg: Seq_Tube N agent: 2
Alg: Seq_Temp_Obs N agent: 2
Alg: S2M2 N agent: 4
Alg: PBS_Tube N agent: 4
Alg: TAHP N agent: 4
Alg: Seq_Tube N agent: 4
Alg: Seq_Temp_Obs N agent: 4
Alg: S2M2 N agent: 6
Alg: PBS_Tube N agent: 6
Alg: TAHP N agent: 6
Alg: Seq_Tube N agent: 6
Alg: Seq_Temp_Obs N agent: 6
Alg: S2M2 N agent: 8
Alg: PBS_Tube N agent: 8
Alg: TAHP N agent: 8
Alg: Seq_Tube N agent: 8
Alg: Seq_Temp_Obs N agent: 8
Alg: S2M2 N agent: 10
Time out for algorithm S2M2 N agent =  10
Alg: PBS_Tube N agent: 10
Alg: TAHP N agent: 10
Alg: Seq_Tube N agent: 10
Alg: Seq_Temp_Obs N agent: 10
Alg: PBS_Tube N agent: 12
Alg: TAHP N agent: 12
Alg: Seq_Tube N agent: 12
Alg: Seq_Temp_Obs N agent: 12
Alg: PBS_Tube N agent: 14
Alg: TAHP N agent: 14
Alg: Seq_Tube N agent: 14
Alg: Seq_Temp_Obs N agent: 14
Alg: PBS_Tube N agent: 16
Time out for algorithm PBS_Tube N agent =  16
Alg: TAHP N agent: 16
Alg: Seq_Tube N agent: 16
Alg: Seq_Temp_Obs N 

In [32]:
from panav.viz import draw_hybrid
# Save checkpoint data
with open("./data/{}_misc.pkl".format(env_name),'wb') as fp:
    pkl.dump({"TIMEOUT":TIMEOUT,"HG":HG},fp)

# draw_hybrid(HG)